In [886]:
import pandas as pd

open_line_list = pd.read_csv("COVID19_open_line_list.csv") # People infected
covid_19       = pd.read_csv("covid_19_data.csv")
#line_list      = pd.read_csv("COVID19_line_list_data.csv")
#nCov           = pd.read_csv("2019_nCoV_data.csv") old

recovered      = pd.read_csv("time_series_covid_19_recovered.csv") # mere overskuelig
deaths         = pd.read_csv("time_series_covid_19_deaths.csv")
confirmed      = pd.read_csv("time_series_covid_19_confirmed.csv")

In [887]:
#gdp           = pd.read_csv("CountriesWGDP.txt").drop(index="GDP values for each country:")
sars = pd.read_csv("sars_2003_complete_dataset_clean.csv")
gdp  = pd.read_csv("GDPbyCountry.csv", skiprows = range(0,4))[['Country Name', 'Country Code', '2016']]

# Cleanup codes

In [888]:
country_codes = gdp[['Country Name', 'Country Code']]
country_codes.rename(columns={'Country Name':'CountryName', 'Country Code':'CountryCode' }, inplace=True)

/Users/Mac/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [889]:
for i, row in country_codes.iterrows():
    name = row['CountryName']
    code = row['CountryCode']
    if "Hong" in name:
        country_codes.at[i, 'CountryName']='Hong Kong'
    if "Egypt" in name:
        country_codes.at[i, 'CountryName']='Egypt'
    if "Iran" in name:
        country_codes.at[i, 'CountryName']='Iran'
    if "Macedonia" in name:
        country_codes.at[i, 'CountryName']='North Macedonia'
    if "Russia" in name:
        country_codes.at[i, 'CountryName']='Russia'
    if "Korea, Rep." in name:
        country_codes.at[i, 'CountryName']='South Korea'

# NaN

In [890]:
#gdp[gdp.isna().any(axis=1)]

# World map

In [891]:
covid_19 = covid_19[covid_19.SNo != 3247]

In [892]:
covid_last = covid_19[covid_19.ObservationDate == '03/04/2020'].drop(columns=['Last Update'])

total      = sum(covid_last.Confirmed)
max_cases  = max(covid_last.Confirmed)
max_cases/total

0.713126734309136

In [893]:
#covid_last[covid_last.Confirmed > 100]

In [894]:
c0 = sum(covid_last[covid_last['Country/Region'] == 'Mainland China'].Confirmed)
d0 = sum(covid_last[covid_last['Country/Region'] == 'Mainland China'].Deaths)
r0 = sum(covid_last[covid_last['Country/Region'] == 'Mainland China'].Recovered)
china = covid_last[covid_last['Country/Region'] == 'Mainland China'][:1]
china['Country/Region'] = 'China'
china['Confirmed']      = c0
china['Deaths']         = d0
china['Recovered']      = r0

c1 = sum(covid_last[covid_last['Country/Region'] == 'US'].Confirmed)
d1 = sum(covid_last[covid_last['Country/Region'] == 'US'].Deaths)
r1 = sum(covid_last[covid_last['Country/Region'] == 'US'].Recovered)
us = covid_last[covid_last['Country/Region'] == 'US'][:1]
us['Confirmed']      = c1
us['Deaths']         = d1
us['Recovered']      = r1

c2 = sum(covid_last[covid_last['Country/Region'] == 'Australia'].Confirmed)
d2 = sum(covid_last[covid_last['Country/Region'] == 'Australia'].Deaths)
r2 = sum(covid_last[covid_last['Country/Region'] == 'Australia'].Recovered)
aus = covid_last[covid_last['Country/Region'] == 'Australia'][:1]
aus['Confirmed']      = c2
aus['Deaths']         = d2
aus['Recovered']      = r2

c3 = sum(covid_last[covid_last['Country/Region'] == 'Canada'].Confirmed)
d3 = sum(covid_last[covid_last['Country/Region'] == 'Canada'].Deaths)
r3 = sum(covid_last[covid_last['Country/Region'] == 'Canada'].Recovered)
can = covid_last[covid_last['Country/Region'] == 'Canada'][:1]
can['Confirmed']      = c3
can['Deaths']         = d3
can['Recovered']      = r3

temp = covid_last[covid_last['Country/Region'] != 'Mainland China']
temp = temp[temp['Country/Region'] != 'US']
temp = temp[temp['Country/Region'] != 'Australia']
temp = temp[temp['Country/Region'] != 'Canada']
new = pd.concat([temp, china, us, aus, can])
new.rename(columns={'Country/Region':'CountryName'}, inplace=True)

In [895]:
merged = pd.merge(new, country_codes, on='CountryName', how='left')
# merged[merged.isna().any(axis=1)]
merged.at[10, 'CountryCode']='GBR'
merged.at[16, 'CountryCode']='TWN'
merged.at[34, 'CountryCode']='MAC'
merged.at[52, 'CountryCode']='BLM'
merged.at[82, 'CountryCode']='USA'

In [896]:
merged.to_csv("../Visualisations/WorldMap_spatial/mapdata.csv")

# Wordcloud

In [897]:

from sklearn import preprocessing
normalized_X = preprocessing.normalize((merged['Confirmed'].values).reshape(1,-1))
merged['new'] = normalized_X[0]*100000
d = {}
import math
for i, row in merged[['CountryName', 'new']].iterrows():
    c = row['CountryName']
    cases = row['new']
    d[c] = cases

# Copy and paste d into vis
# d

# StackedHBChart

In [898]:
#World
bchart = merged[['CountryName', 'Confirmed', 'Deaths', 'Recovered']]
bchart = bchart[bchart.Confirmed > 100]
bchart.rename(columns={'CountryName':'Country'}, inplace=True)
bchart.to_csv("../Visualisations/StackedHorBarChart_quantitative/barchartdata.csv",
                                                              index=False)

In [899]:
china = covid_last[covid_last['Country/Region'] == 'Mainland China']
china = china[['Province/State', 'Confirmed', 'Deaths', 'Recovered']]
china.rename(columns={'Province/State':'Country'}, inplace=True)
china.to_csv("../Visualisations/StackedHorBarChart_quantitative/chinachartdata.csv",
                                                              index=False)

# Evolution - temporal

In [900]:
covid_19.rename(columns={'Province/State':'Province', 'Country/Region':'Country'}, inplace=True)
countries = covid_19[covid_19.Confirmed > 35]['Country'].unique()
dates = covid_19.ObservationDate.unique()

df = pd.DataFrame(dates ,columns=['ObservationDate'])
for country in countries:
    evol = pd.DataFrame()
    for d in dates:
        temp = covid_19[covid_19.ObservationDate == d][['Country', 'Confirmed', 'ObservationDate']]
        china = temp[temp['Country'] == country][:1]
        c0 = sum(temp[temp['Country'] == country].Confirmed)
        china[country] = c0
        evol = pd.concat([evol, china[[country, 'ObservationDate']]])
    df = pd.merge(df, evol, on='ObservationDate', how='left')
df.rename(columns={'Mainland China':'China'}, inplace=True)
df.rename(columns={'South Korea':'SouthKorea'}, inplace=True)
df.rename(columns={'Hong Kong':'HongKong'}, inplace=True)
df = df.drop(columns=['Others']).fillna(0)

In [901]:
df.to_csv("../Visualisations/LineGraph_temporal/linedata.csv", index=False)
df['type'] = 'val1'
df.rename(columns={'ObservationDate':'date'}, inplace=True)
df.to_csv("../Visualisations/LineGraph_temporal2/linedata.csv", index=False)

# Open Line List

In [931]:
dd = ['died', 'death']
dead = open_line_list[open_line_list['outcome'].isin(dd)][['age']].dropna()
dead.rename(columns={'age':'dead'}, inplace=True)
dead.at[1848, 'dead'] = 85
dead.at[10972, 'dead'] = 85
dead.at[11015, 'dead'] = 69

al = ['discharged', 'discharge']
alive = open_line_list[open_line_list['outcome'].isin(al)][['age']].dropna()
alive = alive.drop([5074, 10621, 6621])
alive.rename(columns={'age':'alive'}, inplace=True)


In [933]:
# Print this and copy list(map(int, alive.alive.to_list()))

[44,
 29,
 39,
 46,
 27,
 40,
 30,
 32,
 66,
 28,
 40,
 51,
 56,
 42,
 38,
 7,
 33,
 36,
 4,
 38,
 29,
 23,
 25,
 31,
 47,
 43,
 73,
 25,
 31,
 20,
 94,
 38,
 68,
 29,
 63,
 30,
 28,
 32,
 42,
 49,
 16,
 60,
 53,
 1,
 29,
 29,
 55,
 50,
 51,
 2,
 35,
 34,
 46,
 30,
 55,
 61,
 44,
 61,
 40,
 43,
 35,
 32,
 33,
 58]

In [934]:
# Print this and copy list(map(int, dead.dead.to_list()))

[39, 82, 85, 44, 73, 85, 69, 77, 78, 68, 60]

In [904]:
#open_line_list[open_line_list[['symptoms']].isna().any(axis=1)]

# Correlation

# Sars

In [880]:

merged.rename(columns={'CountryName':'Country'}, inplace=True)
cov = merged[['Confirmed', 'Country']]
sar = sars[sars.Date == "2003-07-11"][['Cumulative number of case(s)', 'Country']]
sar.at[2537, 'Country'] = 'Vietnam'
sar.at[2536, 'Country'] = 'US'
sar.at[2535, 'Country'] = 'UK'
sar.at[2528, 'Country'] = 'Russia'
sar.at[2526, 'Country'] = 'South Korea'
sar.at[2525, 'Country'] = 'Ireland'
sar.at[2512, 'Country'] = 'Taiwan'
sar.at[2511, 'Country'] = 'Macau'
sar.at[2510, 'Country'] = 'Hong Kong'
diseases = pd.merge(cov, sar, on='Country', how='left').dropna()

In [884]:
ssa = sum(diseases['Confirmed'])
scov = sum(diseases['Cumulative number of case(s)'])
ll = []
for i, r in diseases.iterrows():
    sa = int(r['Cumulative number of case(s)'])/scov
    cov = int(r['Confirmed'])/ssa
    c = r['Country']
    d = {'categorie': c, 'values': [{'value': cov, 'rate': "Covid-19"}, {'value': sa, 'rate': "SARS"}]}
    ll.append(d)

In [885]:
import json
with open('../Visualisations/SarsCovid/data2.json', 'w') as fp:
    json.dump(ll, fp)

In [872]:
diseases.to_json('../Visualisations/SarsCovid/data2.json')

In [870]:
#import matplotlib.pyplot as plt
#plt.scatter(diseases['Confirmed'], diseases['Cumulative number of case(s)'])
#plt.ylim([0,2000])

# GDP

In [844]:
merged.rename(columns={'CountryName':'Country'}, inplace=True)
gdp.rename(columns={'Country Name':'Country', '2016':'gdp'}, inplace=True)
cov2 = merged[['Confirmed', 'Country']]
gdp2 = gdp[['Country', 'gdp']]

gdp2.at[124, 'Country'] = 'South Korea'
gdp2.at[110, 'Country'] = 'Iran'
gdp2.at[144, 'Country'] = 'Macau'
gdp2.at[94, 'Country'] = 'Hong Kong'
gdp2.at[79, 'Country'] = 'UK'
gdp2.at[155, 'Country'] = 'North Macedonia'
gdp2.at[249, 'Country'] = 'US'
gdp2.at[200, 'Country'] = 'Russia'
gdp2.at[65, 'Country'] = 'Egypt'
g = pd.merge(cov2, gdp2, on='Country', how='left')
g.at[16,'gdp'] = 589906*10**6
g = g.dropna()

In [845]:
g.to_csv('../Visualisations/Scatter/gdpdata.csv', index=False)